Upload data to sqlite tables.

In [1]:
import pandas as pd
import numpy as np
from IPython import display as dis
import scipy.io.wavfile as wav
import tensorflow as tf
from tensorflow.contrib.legacy_seq2seq.python.ops.seq2seq import basic_rnn_seq2seq
from tensorflow.contrib.rnn import RNNCell, LSTMCell, MultiRNNCell
from scipy import signal 
from librosa import core
import time

%matplotlib inline

In [2]:
dis.Audio("dataset/wav/Ses01F_impro01/Ses01F_impro01_F000.wav")

In [23]:
(sig,rate) = core.load("dataset/wav/Ses01F_impro01/Ses01F_impro01_F000.wav", sr = 8000)
print(sig, rate)
print(len(sig))

[-0.00372308 -0.00450607 -0.00213044 ..., -0.00266384 -0.00192541  0.        ] 8000
15565


In [24]:
dis.Audio(data = sig, rate = rate)

In [25]:
time_step = 15565
hidden_layers = 1
batch_size = 1000

sig = np.reshape(sig[:-(time_step%batch_size)], (batch_size,-1, 1)) #clipping out the remaining signal that cannot be fully accomodated in the batch
print(sig.shape)

(1000, 15, 1)


In [26]:

class network(object):
    
    def __init__(self):
        pass
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        global time_step 
        global batch_size 
        global hidden_layers
        
        
        with tf.variable_scope("Input"):
            self.x_input = tf.placeholder("float", shape=[1, int(time_step/batch_size), 1], name='x')
            self.y_input_ = tf.placeholder("float", shape=[1, int(time_step/batch_size), 1], name='y')
            
            self.keep_prob = tf.placeholder("float", name='kp')
            self.lr = tf.placeholder("float", name='lr')
            self.hidden = tf.placeholder("float", shape=[1, 1], name='hidden')
            self.current = tf.placeholder("float", shape=[1, 1], name='current')
            self.state = self.hidden, self.current
            
            self.x_list = tf.unstack(self.x_input, axis= 1)
            self.y_list_ = tf.unstack(self.y_input_, axis = 1)
            
            self.y_list = []
            loss = 0
            
        with tf.variable_scope("lstm"):
            lstm = tf.contrib.rnn.BasicLSTMCell(1)
            #multi_cell = MultiRNNCell([lstm for i in range(hidden_layers)] )
            
            #self.y, states = basic_rnn_seq2seq(self.x_list, self.y_list_, multi_cell)
            
            #state = tf.convert_to_tensor([np.zeros([batch_size, s]) for s in lstm.state_size])
            
            for c, sig in enumerate(self.x_list):
                output, self.state = lstm(sig, self.state)
                loss += tf.losses.mean_squared_error(output, sig)
                self.y_list.append(output)
                
            #self.y = tf.slice(self.y, [0, 0], [-1,2])
            #self.out = tf.squeeze(self.y)
            #self.y = tf.layers.dense(self.y[0], classes, activation = None)
            #self.y = tf.slice(self.y[0], [0, 0], [-1,2])
            
        with tf.variable_scope("Loss"):
            
            self.pred = tf.stack(self.y_list)
            
            self.regularized_loss = loss #tf.losses.mean_squared_error(self.pred, self.y_input_)
            
        with tf.variable_scope("Optimizer"):
            learning_rate=self.lr
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        
        # add Saver ops
        self.saver = tf.train.Saver()
        

In [27]:
sigs = []
for i in range(batch_size):
    sigs.append((sig[i,...])[np.newaxis,:])
#for s in sigs:
#    print(s.shape)

In [28]:
import collections

y_pred_list = []
final_state = None
class Train:    
    
    def train(epochs, net, lrs):
        global final_state
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            start_time = time.perf_counter()
            
            hidden_state = np.zeros([1, 1])
            current_state = np.zeros([1, 1])
            state = hidden_state, current_state
            
            for c, lr in enumerate(lrs):
                for epoch in range(1, (epochs+1)):
                    print("Step {} ".format(epoch))
                    for s in sigs:
                        x_train = y_train = s
                        
                        _, train_loss,state, y_pred = sess.run([net.train_op, net.regularized_loss, net.state, net.pred], #net.summary_op
                                                  feed_dict={net.x_input: x_train, 
                                                             net.y_input_: y_train,
                                                             net.keep_prob:1,
                                                             net.hidden:state[0],
                                                             net.current:state[1],
                                                             net.lr:lr})
                        y_pred_list.append(y_pred)
                        final_state = state

                        print("Training Loss: {:.6f}".format(train_loss))

                        #valid_accuracy,valid_loss = sess.run([net.tf_accuracy, net.regularized_loss], #net.summary_op 
                        #                                      feed_dict={net.x_input: x_valid[np.newaxis,...], 
                        #                                                 net.y_input_: y_valid[np.newaxis,...],
                        #                                                 net.lr:lr})

                        #accuracy, y_pred = sess.run([net.regularized_loss, 
                        #                           net.pred], 
                        #                            feed_dict={net.x_input: x_train, 
                        #                                     net.y_input_: y_train,
                        #                                     net.hidden:state[0],
                        #                                     net.current:state[1],
                        #                                     net.lr:lr})



In [29]:
import itertools

class Hyperparameters:
    def start_training():
        
        epochs = 1
        lrs = [1e-5]
        n = network()
        n.build_layers()
        Train.train(epochs, n, lrs)
            

In [30]:
x_train = y_train = sig
Hyperparameters.start_training()

Step 1 
Training Loss: 0.000052
Training Loss: 0.000060
Training Loss: 0.000037
Training Loss: 0.000078
Training Loss: 0.000075
Training Loss: 0.000177
Training Loss: 0.000127
Training Loss: 0.000176
Training Loss: 0.000140
Training Loss: 0.000132
Training Loss: 0.000108
Training Loss: 0.000044
Training Loss: 0.000170
Training Loss: 0.000210
Training Loss: 0.000205
Training Loss: 0.000091
Training Loss: 0.000035
Training Loss: 0.000093
Training Loss: 0.000037
Training Loss: 0.000107
Training Loss: 0.000056
Training Loss: 0.000049
Training Loss: 0.000054
Training Loss: 0.000101
Training Loss: 0.000040
Training Loss: 0.000043
Training Loss: 0.000115
Training Loss: 0.000055
Training Loss: 0.000047
Training Loss: 0.000057
Training Loss: 0.000029
Training Loss: 0.000027
Training Loss: 0.000136
Training Loss: 0.000103
Training Loss: 0.000068
Training Loss: 0.000057
Training Loss: 0.000021
Training Loss: 0.000043
Training Loss: 0.000043
Training Loss: 0.000019
Training Loss: 0.000015
Training

Training Loss: 0.018284
Training Loss: 0.006521
Training Loss: 0.017459
Training Loss: 0.006579
Training Loss: 0.005215
Training Loss: 0.020614
Training Loss: 0.004709
Training Loss: 0.021516
Training Loss: 0.006084
Training Loss: 0.009336
Training Loss: 0.022760
Training Loss: 0.005721
Training Loss: 0.025692
Training Loss: 0.005735
Training Loss: 0.013695
Training Loss: 0.026859
Training Loss: 0.006810
Training Loss: 0.027573
Training Loss: 0.007180
Training Loss: 0.016065
Training Loss: 0.027144
Training Loss: 0.006761
Training Loss: 0.023708
Training Loss: 0.008036
Training Loss: 0.015684
Training Loss: 0.017255
Training Loss: 0.006565
Training Loss: 0.020813
Training Loss: 0.004228
Training Loss: 0.010498
Training Loss: 0.014585
Training Loss: 0.005138
Training Loss: 0.013161
Training Loss: 0.003744
Training Loss: 0.003822
Training Loss: 0.009975
Training Loss: 0.004117
Training Loss: 0.004995
Training Loss: 0.005257
Training Loss: 0.000821
Training Loss: 0.003921
Training Loss: 0

Training Loss: 0.000063
Training Loss: 0.000056
Training Loss: 0.000051
Training Loss: 0.000044
Training Loss: 0.000050
Training Loss: 0.000049
Training Loss: 0.000020
Training Loss: 0.000043
Training Loss: 0.000044
Training Loss: 0.000029
Training Loss: 0.000029
Training Loss: 0.000053
Training Loss: 0.000013
Training Loss: 0.000020
Training Loss: 0.000058
Training Loss: 0.000046
Training Loss: 0.000043
Training Loss: 0.000041
Training Loss: 0.000043
Training Loss: 0.000040
Training Loss: 0.000016
Training Loss: 0.000047
Training Loss: 0.000025
Training Loss: 0.000070
Training Loss: 0.000111
Training Loss: 0.000039
Training Loss: 0.000067
Training Loss: 0.000019
Training Loss: 0.000012
Training Loss: 0.000019
Training Loss: 0.000041
Training Loss: 0.000023
Training Loss: 0.000045
Training Loss: 0.000036
Training Loss: 0.000045
Training Loss: 0.000098
Training Loss: 0.000076
Training Loss: 0.000036
Training Loss: 0.000008
Training Loss: 0.000080
Training Loss: 0.000045
Training Loss: 0

In [31]:
for y in y_pred_list:
    print(y.shape)

(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)
(15, 1, 1)

In [32]:
y = np.concatenate(y_pred_list)

In [33]:
y.shape

(15000, 1, 1)

In [34]:
y = np.squeeze(y)
y.shape

(15000,)

In [15]:
dis.Audio(data = y, rate = rate)

In [16]:
final_state

LSTMStateTuple(c=array([[ 0.0070931]], dtype=float32), h=array([[ 0.003554]], dtype=float32))